In [ ]:
# izracunati fingerprints, izbaciti kojima je varijanca 0, ubaciti u model, usporediti rezultate s prijasnjima

In [ ]:
import pandas as pd 
import numpy as np 
from rdkit import Chem 
from rdkit.Chem import AllChem, Draw, Descriptors
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
data = pd.read_csv('home/ivana/repozitorij/QSAR_Kristina_topljivost/data/solubility_all_new_data.csv', index_col=0)

In [ ]:
data

In [ ]:
# Pretvaranje SMILES iz DataFrame-a u listu molekula
smiles_list = data['smiles'].to_list()

fps = []  # Lista koja će sadržavati otiske svih molekula

for s in smiles_list:
    mol = Chem.MolFromSmiles(s)
    if mol:  # Provjera da li je molekula valjano konvertirana
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
        fps.append(list(fp.ToBitString()))  # Dodavanje otiska u listu

# 'fps' sadrži otiske svih molekula
for fp in fps:
    print(fp)

In [ ]:
#data['fingerprints'] = fps

In [ ]:
# Pretvaranje liste otisaka u DataFrame
fp_df = pd.DataFrame(fps)
print(data.shape, fp_df.shape)

# Spajanje otisaka s originalnim podacima
data_fp = pd.concat([data, fp_df], axis=1)

In [ ]:
data_fp.iloc[:, 7:]

In [ ]:
data_fp.iloc[:, 7:] = data_fp.iloc[:, 7:].astype(int)

In [ ]:
# Izolirajte kolone otisaka
X = data_fp.iloc[:, 7:] 

# Izračunajte varijancu za svaku kolonu
variances = X.var()

# Pronađite kolone koje imaju varijancu veću od 1e-2
high_variance_cols = variances[variances > 1e-2].index
print(high_variance_cols)

# Filtrirajte podatke da zadržite samo te kolone
data_fp.iloc[:, 7:]=data_fp[high_variance_cols]
print(data_fp)

# Filtrirajte podatke da zadržite samo te kolone
final_data = pd.concat([data_fp.iloc[:, :7], data_fp[high_variance_cols]], axis=1)
print(final_data)


In [ ]:
train_set,test_set=train_test_split(final_data,test_size=.2,random_state=42) 
train_set.shape, test_set.shape

In [ ]:
# Odabir značajki i ciljne varijable
#features = ['molwt', 'logp', 'balabanj', 'tpsa']
#features = [final_data.iloc[0,3:]]
features = final_data.columns[3:]
target = 'Solubility'

# Podijela podataka na skupove za treniranje, validaciju i testiranje
X_train, X_test, y_train, y_test = train_test_split(final_data[features], final_data[target], test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Treniranje XGBoost modela s ranom zaustavom
model = xgb.XGBRegressor(n_estimators=500, learning_rate=0.15, max_depth=3)
model.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_valid, y_valid)], verbose=False)

# Predviđanje na testnom skupu
y_pred = model.predict(X_test)

# Izračunavanje srednjekvadratne pogreške
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

# Izračunavanje R2 koeficijenta determinacije
r2 = r2_score(y_test, y_pred)
print('R2 Score:', r2)

In [ ]:
import matplotlib.pyplot as plt

# Predviđanje na testnom skupu
y_pred = model.predict(X_test)

# Plotanje dijagrama raspršenosti
plt.scatter(y_pred, y_test, c='red', label='Predviđene vrijednosti')
plt.scatter(y_test, y_test, c='green', label='Stvarne vrijednosti')
plt.xlabel('Predviđene vrijednosti')
plt.ylabel('Stvarne vrijednosti')
plt.title('Dijagram raspršenosti - Stvarne vs Predviđene vrijednosti')

plt.legend()
plt.show()

In [ ]:
# Izdvajanje značajki (features) iz dataframe-a
features_data = final_data.iloc[:100, 3:]
print(type(features_data))

# Izračun matrice korelacije
corr_matrix = features_data.corr().abs()
print(corr_matrix)

In [ ]:
import seaborn as sns

# Izdvajanje značajki (features) iz dataframe-a
features_data = final_data.iloc[:500, 3:]

# Izračun matrice korelacije
corr_matrix = features_data.corr()

# Postavljanje veličine grafikona
plt.figure(figsize=(15, 12))  # Možete prilagoditi veličinu prema potrebi

# Crtanje heatmap
sns.heatmap(corr_matrix, cmap='coolwarm', vmin=-1, vmax=1)

# Prikaz grafikona
plt.title("Matrica korelacije")
plt.show()


In [ ]:
# Selektiranje gornjeg trokuta matrice korelacije
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
#print(upper)

# Pronalaženje indeksa značajki koje koreliraju više od 0.99
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
print(to_drop)

# Uklanjanje značajki iz originalnog dataframe-a
#data_99=final_data.drop(to_drop, axis=1, inplace=True)
#print(data_99)

In [ ]:
#from rdkit.Chem import AllChem, rdmolops

## Pretvaranje SMILES-a u RDKit molekule
#data['mol'] = data['smiles'].apply(Chem.MolFromSmiles)

## Izračun Morganovih otisaka
#data['morgan_fp'] = data['mol'].apply(lambda x: AllChem.GetMorganFingerprintAsBitVect(x, 2, nBits=1024))

## Izračun RDKit otisaka
#data['rdkit_fp'] = data['mol'].apply(lambda x: rdmolops.RDKFingerprint(x))

# #Pretvaranje otisaka u DataFrame
#morgan_df = data['morgan_fp'].apply(lambda fp: pd.Series(list(fp.ToBitString()))).astype(int)
#rdkit_df = data['rdkit_fp'].apply(lambda fp: pd.Series(list(fp.ToBitString()))).astype(int)


In [ ]:
#morgan_variances = morgan_df.var()
#rdkit_variances = rdkit_df.var()

#zero_variance_morgan = morgan_variances[morgan_variances == 0].index.tolist()
#zero_variance_rdkit = rdkit_variances[rdkit_variances == 0].index.tolist()

#print(f"Number of bits with zero variance in Morgan fingerprints: {len(zero_variance_morgan)}")
#print(f"Number of bits with zero variance in RDKit fingerprints: {len(zero_variance_rdkit)}")
